## DASH APP

In [18]:

# --- Lancer ---
if __name__ == "__main__":
    import webbrowser
    url = "http://127.0.0.1:8050/"
    webbrowser.open_new_tab(url)
    app.run(debug=True, port=8050, threaded=True)


### V2

In [64]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import json
import numpy as np
from pathlib import Path
from datetime import datetime
import sys
import importlib
import ipynbname 


code_path = ipynbname.path().parent.parent
# Ajouter le dossier scripts au path
scripts_path = code_path / "scripts"
base_path = code_path.parent
sys.path.append(str(scripts_path.resolve()))

import data_utils
importlib.reload(data_utils)
from data_utils import import_data_raw, import_data_sig


# --- Fonctions utilitaires ---
def symlog(x):
    """Logarithme symétrique : log10(|x|+1) * signe(x)"""
    if pd.isna(x):
        return None
    return np.sign(x) * np.log10(abs(x) + 1)


# --- Charger données ---
filename = "data_final_all_norm.csv"
filepath = base_path / "Data" / 'data_final' / filename
df_data = pd.read_csv(filepath)

# Global SIG
gdf_world = import_data_sig(base_path,'world.geojson')


# --- Year options ---
year_options = sorted(df_data['Year'].unique())
first_year = 1960  # first year for the slider
last_year = max(year_options)


# --- Normalisations et colonnes d'unité ---
norm_map = {
    'No norm': ('Value', 'Unit'),
    'Area': ('Value_norm_area', 'Unit_norm_area'),
    'Population': ('Value_norm_population', 'Unit_norm_population'),
    #'Hab/km2': ('Value_norm_densite', 'Unit_norm_densite'),
    'GDP': ('Value_norm_gdp', 'Unit_norm_gdp'),
    'GDP(PPP)': ('Value_norm_ppp', 'Unit_norm_ppp'),
    'GDP/hab': ('Value_norm_gdp_hab', 'Unit_norm_gdp_hab'),
    'PGD(PPP)/hab': ('Value_norm_ppp_hab', 'Unit_norm_ppp_hab'),
}


def simplify_geom(geom, tol=0.1):
    if geom is None:
        return None
    if geom.geom_type == 'Polygon':
        return geom.simplify(tol, preserve_topology=True)
    elif geom.geom_type == 'MultiPolygon':
        return type(geom)([poly.simplify(tol, preserve_topology=True) for poly in geom.geoms])
    return geom


gdf_world = gdf_world[gdf_world['Country_code'].notna()].copy()
gdf_world['geometry'] = gdf_world['geometry'].apply(lambda g: simplify_geom(g, tol=0.1))


# --- Dash app ---
app = dash.Dash(__name__)


@app.callback(
    Output("world_map", "figure"),
    Input("indicator", "value"),
    Input("database", "value"),
    Input("type", "value"),          
    Input("year", "value"),
    Input("scale", "value"),
    Input("color_range", "value"),
    Input("normalization", "value")
)
def update_map(indicator, database, type_value, year, scale, color_range, normalization):

    # --- Filter data ---
    df_filtered = df_data[
        (df_data['Indicator'] == indicator)
        & (df_data['Source'] == database)
        & (df_data['Type'] == type_value)
        & (df_data['Year'] == year)
    ].copy()

    # --- Merge with geometry ---
    gdf_merged = gdf_world.merge(df_filtered, on='Country_code', how='left')
    geojson_data = json.loads(gdf_merged.to_json())

    # --- Columns for values and units ---
    col_value, col_unit = norm_map[normalization]
    if col_value not in gdf_merged.columns:
        gdf_merged[col_value] = np.nan
    if col_unit not in gdf_merged.columns:
        gdf_merged[col_unit] = ""

    z_values = gdf_merged[col_value]
    no_data_at_all = z_values.dropna().empty

    # --- No data case ---
    if no_data_at_all:
        z_plot_scaled = np.zeros(len(gdf_merged))
        colorscale_to_use = [[0, 'lightgray'], [1, 'lightgray']]
        zmin, zmax = 0, 1
        colorbar_ticks = dict(ticks="", tickvals=[], ticktext=[])
        hovertemplate = f"<b>%{{text}}</b><br>{indicator} ({normalization}) = No data<extra></extra>"
    else:
        # --- Determine color range ---
        colorscale_to_use = 'RdYlGn_r'
        if color_range == 'raw':
            zmin, zmax = z_values.min(), z_values.max()
        elif color_range.startswith("q"):
            q_low = float(color_range[1:])
            q_high = 1 - q_low
            zmin, zmax = z_values.quantile(q_low), z_values.quantile(q_high)
        elif color_range.startswith("*"):
            factor = float(color_range[1:])
            zmin, zmax = factor * z_values.min(), factor * z_values.max()
        else:
            raise ValueError("color_range must be 'raw', 'q0.xx', or '*0.xx'")

        # --- Apply scale ---
        if scale == 'rank':
            colorscale_to_use = 'YlOrBr'
            z_plot = z_values.rank(ascending=True)
            zmin, zmax = z_plot.min(), z_plot.max()
        elif scale == 'absolute':
            z_plot = z_values.copy()
            zborne = max(abs(zmin), abs(zmax))
            zmin, zmax = -zborne, zborne
        elif scale == 'relative':
            z_plot = z_values.copy()
        elif scale == 'log':
            z_plot = z_values.apply(symlog)
        else:
            raise ValueError("scale must be 'absolute', 'relative', 'rank', or 'log'")

        # --- Dynamic unit scaling for large numbers ---
        unit_prefix = gdf_merged[col_unit].iloc[0] if col_unit in gdf_merged.columns else ""
        unit_multiplier = 1
        if scale in ['absolute', 'relative']:
            vmax_val = z_plot.max()
            if vmax_val >= 1e9:
                unit_multiplier = 1e9
                unit_prefix = f"G{unit_prefix}"
            elif vmax_val >= 1e6:
                unit_multiplier = 1e6
                unit_prefix = f"M{unit_prefix}"
            elif vmax_val >= 1e3:
                unit_multiplier = 1e3
                unit_prefix = f"k{unit_prefix}"
        
        # Append "/year" if type is annual
        if type_value == "Annual":
            unit_prefix = f"{unit_prefix}/year"
        
        # --- Scale for plotting & hover ---
        z_plot_scaled = z_plot / unit_multiplier
        
        # --- Colorbar ticks for absolute scale ---
        if scale in ['absolute', 'relative']:
            if scale == 'absolute':
                # 3 ticks entre 0 et vborne, et 3 entre -vborne et 0
                pos_ticks = np.linspace(0, zborne / unit_multiplier, 4 + 1)[1:]  # exclure 0
                neg_ticks = np.linspace(-zborne / unit_multiplier, 0, 4 + 1)[:-1]  # exclure 0
                tickvals = np.concatenate([neg_ticks, [0], pos_ticks])
            else:
                # relative scale: linspace normal
                tickvals = np.linspace(z_plot_scaled.min(), z_plot_scaled.max(), 8)
        
            # --- Tick labels with smart rounding ---
            ticktext = []
            for v in tickvals:
                abs_v = abs(v)
                if abs_v > 100:
                    ticktext.append(f"{int(round(v))}")
                elif abs_v > 10:
                    ticktext.append(f"{round(v, 1)}")
                else:
                    ticktext.append(f"{round(v, 2)}")
            
            colorbar_ticks = dict(tickvals=tickvals, ticktext=ticktext)
        else:
            colorbar_ticks = {}


        # --- Hovertemplate ---
        hovertemplate = f"<b>%{{text}}</b><br>{indicator} ({normalization}) = %{{customdata[0]:.2f}} {unit_prefix}<extra></extra>"

    # --- Choropleth figure ---
    fig = go.Figure(go.Choropleth(
        geojson=geojson_data,
        locations=gdf_merged.index,
        z=z_plot_scaled,
        text=gdf_merged['name'],
        colorscale=colorscale_to_use,
        zmin=zmin/unit_multiplier if 'unit_multiplier' in locals() else zmin,
        zmax=zmax/unit_multiplier if 'unit_multiplier' in locals() else zmax,
        zmid=0 if (not no_data_at_all and scale == 'absolute') else None,
        customdata=np.stack([z_plot_scaled], axis=-1),
        hovertemplate=hovertemplate,
        colorbar=dict(
            title=dict(
                text=f"<b>{unit_prefix}</b>",
                side="top",
                font=dict(size=14, color="black", family="Arial"),
            ),
            x=0.0,
            xanchor='left',
            len=0.8,
            thickness=30,
            tickvals=colorbar_ticks.get('tickvals', None),
            ticktext=colorbar_ticks.get('ticktext', None)
        )
    ))

    # --- Layout ---
    fig.update_layout(
        geo=dict(
            scope="world", projection_type="natural earth",
            showcountries=True, showcoastlines=True, showland=True, showocean=True,
            landcolor="lightgray", oceancolor="lightblue", lakecolor="lightblue",
            domain=dict(x=[0.07, 1], y=[0, 1])
        ),
        margin=dict(l=0, r=0, t=0, b=0),
        shapes=[
            dict(
                type="rect",
                xref="paper", yref="paper",
                x0=0.0, x1=0.02,
                y0=0.92, y1=0.95,
                fillcolor="lightgray",
                line=dict(color="black", width=1),
                layer="above"
            )
        ],
        annotations=[
            dict(
                x=0.03, y=0.935,
                xref="paper", yref="paper",
                text=f'<b>No Data<b>',
                showarrow=False,
                xanchor="left",
                yanchor="middle",
                font=dict(color="black", size=12)
            )
        ]
    )

    return fig


# --- Options pour les menus ---
indicator_options = df_data['Indicator'].unique()
database_options = df_data['Source'].unique()
scale_options = ['absolute', 'relative', 'rank', 'log']
color_range_options = ['raw', 'q0.01', 'q0.05', 'q0.1', '*0.8']
norm_map_labels = list(norm_map.keys())


# --- Layout ---
app.layout = html.Div([
    html.Div([
        # Sidebar for controls
        html.Div([
            html.Label("Indicator"),
            dcc.Dropdown(
                id="indicator",
                options=[{"label": i, "value": i} for i in indicator_options],
                value=indicator_options[0],
                placeholder="Indicator",
                style={'marginBottom': '20px'}
            ),
            html.Label("Database"),
            dcc.Dropdown(
                id="database",
                options=[{"label": i, "value": i} for i in database_options],
                value=database_options[0],
                placeholder="Database",
                style={'marginBottom': '20px'}
            ),
            html.Label("Normalization"),
            dcc.Dropdown(
                id="normalization",
                options=[{"label": name, "value": name} for name in norm_map_labels],
                value=norm_map_labels[0],
                placeholder="Normalization",
                style={'marginBottom': '20px'}
            ),
            html.Label("Type"),
                dcc.Dropdown(
                    id="type",
                    options=[
                        {"label": "Annual", "value": "Annual"},
                        {"label": "Cumulative", "value": "Cumulative"}
                    ],
                    value="Annual",
                    placeholder="Type",
                    style={'marginBottom': '20px'}
                ),
            html.Label("Scale"),
            dcc.Dropdown(
                id="scale",
                options=[{"label": i, "value": i} for i in scale_options],
                value='relative',
                placeholder="Scale",
                style={'marginBottom': '20px'}
            ),
            html.Label("Color Range"),
            dcc.Dropdown(
                id="color_range",
                options=[{"label": i, "value": i} for i in color_range_options],
                value='raw',
                placeholder="Color range",
                style={'marginBottom': '20px'}
            ),
        ], style={
            'flex': '0 0 250px',
            'padding': '15px',
            'backgroundColor': '#f8f9fa',
            'boxShadow': '2px 0px 5px rgba(0,0,0,0.1)',
            'height': '100vh',
            'overflowY': 'auto'
        }),

        # Main content: map + slider
        html.Div([
            html.Div([
                dcc.Graph(
                    id="world_map",
                    style={
                        'height': '85vh',
                        'width': '80vw',
                        'margin': '0 auto'   # center horizontally
                    }
                )
            ], style={
                'display': 'flex',
                'justifyContent': 'center',   # horizontal centering
                'alignItems': 'center',       # vertical centering if needed
                'height': '85vh'
            }),
            html.Div([
                html.Label("Year", style={'fontWeight': 'bold', 'textAlign': 'center', 'display': 'block'}),
                dcc.Slider(
                    id='year',
                    min=first_year,
                    max=last_year,
                    step=1,
                    marks={
                        int(year_options[0]): str(first_year),  # first year
                        int(year_options[-1]): str(last_year),  # last year
                        **{int(y): str(int(y)) for y in year_options if int(y) % 5 == 0 and y >= first_year}  # every 5 years
                    },
                    value=max(year_options),  # initial position at last year
                    tooltip={"placement": "bottom", "always_visible": False},
                )
            ], style={
                'width': '60%',
                'margin': '10px auto 0 auto',  # center horizontally
                'padding': '0'
            })

        ], style={'flex': '1', 'padding': '0px'}),
    ], style={'display': 'flex', 'flexDirection': 'row', 'height': '100vh'}),
])




In [65]:

# --- Lancer ---
if __name__ == "__main__":
    import webbrowser
    url = "http://127.0.0.1:8050/"
    webbrowser.open_new_tab(url)
    app.run(debug=True, port=8050, threaded=True)
